## Sales Scraper for crexi.com (Sales)

 A scraper dedicated to extracting information from Loopnet.com
 During running a brower will open, it's important that the browser opens with a proper resoltion so that all element are loaded.

## ⚠️WARNING⚠️

Overuse of this script may cause the provider to refuse the connection, either temporarily or permanently. To ensure safe usage:
- Avoid running the script continuously all day.
- Keep the randSleep function (present in the first cell) unchanged or increase its values for longer pauses.
- Only use when necessary
- This scrapper is kinda slow, and it's intented to be slow, as to avoid ban from the provider


## SETUP

1. install the dependecies
2. put the chromedriver on the drivers folder
3. all done


## USAGE

Run the first cell to do the setup. A browser will open, Do not close the browser.

Go into crexi.com, login with an account, and go into the results of your search, for example for San Antonio - TX



In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import numpy as np
import random
import json
import pandas as pd

def randSleep():
    time.sleep(random.uniform(5, 7))

def randSleepLong():
    time.sleep(random.uniform(10, 15))   


def pretty_print(d, indent=0):
    for key, value in d.items():
        print("  " * indent + str(key) + ":")
        if isinstance(value, dict):
            pretty_print(value, indent + 1)
        else:
            print("  " * (indent + 1) + str(value))


service = Service("./drivers/chromedriver")
driver = webdriver.Chrome()
driver.page_load_strategy = 'normal'  # Default behavior, waits for complete page load
wait = WebDriverWait(driver, 10) # mistery wait
driver.get("https://google.com/")



With the browser open on the search page, change the results per page to 100. This is done to speed up the link gathering process.

Run the cell below. It will go trought every page, and save the link for each ad on a file ```urls.txt```

In [13]:
def get_cards():
    cards = driver.find_elements(By.CSS_SELECTOR, ".cui-card-cover-link")
    cards_return = []
    for i in range(len(cards)):
        print(i,cards[i].get_attribute("href"))
        cards_return.append(cards[i].get_attribute("href"))
    
    return cards_return

def get_max_page():
    pages = driver.find_elements(By.CSS_SELECTOR, ".pagination.ng-star-inserted")
    pages = just_text(pages, "\n")
    pages = [page for page in pages if page.isdigit()]
    return pages[-1]



first_page = driver.current_url


urls = open("urls.txt", "w")

pages = driver.find_elements(By.CSS_SELECTOR, ".next.ng-star-inserted")
print(len(pages))


nxt_page = driver.find_elements(By.CSS_SELECTOR, ".next.ng-star-inserted")

page = 1


while True:
    print("page: ", page)
    cards = get_cards()
    for j in range(len(cards)):
        urls.write(cards[j] + "\n")
    randSleep()
    #seo-footer ng-star-inserted
    
    nxt_page = driver.find_elements(By.CSS_SELECTOR, ".next.ng-star-inserted")
    if nxt_page == []:
        break
    else:
        nxt_page[0].click()
    randSleepLong()

    page += 1

print("done with link gathering \a")



1
page:  1
0 https://www.crexi.com/properties/1690824/texas-lender-owned-2-hotel-san-antonio-portfolio?recommId=cd75f81443bafcfbee2ae3a5b81a1e3d
1 https://www.crexi.com/properties/1699429/texas-conviva-care-center-san-antonio-tx?recommId=cd75f81443bafcfbee2ae3a5b81a1e3d
2 https://www.crexi.com/properties/1699072/texas-take-5-san-antonio-tx?recommId=cd75f81443bafcfbee2ae3a5b81a1e3d
3 https://www.crexi.com/properties/1695037/texas-kneaders-bakery-cafe?recommId=cd75f81443bafcfbee2ae3a5b81a1e3d
4 https://www.crexi.com/properties/1690900/texas-1127-south-st-marys?recommId=cd75f81443bafcfbee2ae3a5b81a1e3d
5 https://www.crexi.com/properties/1686044/texas-single-tenant-starbucks-with-drive-thru?recommId=cd75f81443bafcfbee2ae3a5b81a1e3d
6 https://www.crexi.com/properties/1680481/texas-frito-lay-distribution?recommId=cd75f81443bafcfbee2ae3a5b81a1e3d
7 https://www.crexi.com/properties/1675404/texas-las-palmas?recommId=cd75f81443bafcfbee2ae3a5b81a1e3d
8 https://www.crexi.com/properties/1668601/tex

The cell below is to load a list of links from a file, like the one created from the cell above

In [14]:
link_list = np.loadtxt("urls.txt", dtype=str)
print(link_list.shape)

(860,)


With the list of links properlly allocated into ```link_list```, run the next cell, and wait.

In [15]:


def get_adress():
    address = driver.find_elements(By.CSS_SELECTOR, ".address-line")
    address = address[0].text.split("\n")[0]
    return address

def just_text(element,splitter): 
    text = ""
    for e in element:
        text  += e.text
    text_array = text.split(splitter)
    return text_array


def get_contacts(): #
    broker_box = driver.find_elements(By.CSS_SELECTOR, ".broker-data")

    # if there is a see all button, click it
    names = []
    phones = []
    licenses = []
    if len(broker_box) == 0:
        return names, phones, licenses
    
    see_all = broker_box[0].find_elements(By.CSS_SELECTOR, ".right-control")
    
    if len(see_all) > 0:
        see_all[0].click()
        time.sleep(2)
        page_cover = driver.find_elements(By.CSS_SELECTOR, ".cdk-overlay-container")
        brokers = page_cover[0].find_elements(By.CSS_SELECTOR, ".crxe-broker-item")
        
        for i in range(len(brokers)):
            name = brokers[i].find_elements(By.CSS_SELECTOR, ".name")
            if len(name) == 0:
                name = "-"
            else:
                name = name[0].text.replace("\n", " ")
            phone = brokers[i].find_elements(By.CSS_SELECTOR, ".show-link.phone")
            if len(phone) == 0:
                phone = "-"
            else:
                phone = phone[0].text
            license = brokers[i].find_elements(By.CSS_SELECTOR, ".info.ng-star-inserted")
            if len(license) == 0:
                license = "-"
            else:
                license = license[0].text
                license = license.replace("License: ", "")
            
            names.append(name)
            phones.append(phone)
            licenses.append(license)
        #print("-names: ", names)
        #print("-phones: ", phones)
        #print("-licenses: ", licenses)
    # if there is no click button
    else:
        brokers = driver.find_elements(By.CSS_SELECTOR, ".broker-info")
        if len(brokers) == 0:
            names.append("")
            phones.append("")
            licenses.append("")
        else:
            for i in range(0,len(brokers)):
                names.append(brokers[i].find_elements(By.CSS_SELECTOR, ".name")[0].text.replace("\n", " "))
                phones.append(brokers[i].find_elements(By.CSS_SELECTOR, ".info.ng-star-inserted")[0].text.replace("Phone Number: ", ""))
                licenses.append(brokers[i].find_elements(By.CSS_SELECTOR, ".info")[0].text)
    return names, phones, licenses



def get_details(): # Check
    details = driver.find_elements(By.CSS_SELECTOR, ".text.detail-name")
    details_vals = driver.find_elements(By.CSS_SELECTOR, ".text.detail-value")
    details_names = []
    details_values = []
    for i in range(len(details)):
        details_names.append(details[i].text)
        details_values.append(details_vals[i].text)
        #print(details[i].text, details_vals[i].text)
    return details_names, details_values

def get_price(): # Check
    price_type = driver.find_elements(By.CSS_SELECTOR, ".text.term-name")[0].text.replace("\n", "")
    price = driver.find_elements(By.CSS_SELECTOR, ".text.term-value")[0].text.replace("\n", "")
    return price_type, price

def get_listing():
    info_names = driver.find_elements(By.CSS_SELECTOR, ".pdp_updated-date-label")
    infos = driver.find_elements(By.CSS_SELECTOR, ".pdp_updated-date-value")

    info_type = []
    info_val = []
    for i in range(len(info_names)):
        info_type.append(info_names[i].text.split("\n")[0])
        info_val.append(infos[i].text.split("\n")[0])
    return info_type, info_val

def get_infos():
    return_infos = {}

    return_infos["link"] = driver.current_url
    return_infos["address"] = get_adress()

    price = get_price()
    return_infos[price[0]] = price[1]

    ad_date_type, ad_date_val = get_listing()
    for i in range(len(ad_date_type)):
        return_infos[ad_date_type[i]] = ad_date_val[i]

    details_names, details_values = get_details()
    for i in range(len(details_names)):
        return_infos[details_names[i]] = details_values[i]
    
    return_infos["names"], return_infos["phones"], return_infos["licenses"] = get_contacts()

    return return_infos


loaded_list = link_list
links_done = []

i = 0
datajson = {}
for link in link_list:
    print(i+1,len(link_list),"link: ", link)
    driver.get(link)
    randSleep()
    datajson[i] = get_infos()
    print(datajson[i])
    links_done.append(link)
    loaded_list = np.delete(loaded_list, np.where(loaded_list == link))
    i += 1

with open("data.json", "w") as outfile:
    json.dump(datajson, outfile)






1 860 link:  https://www.crexi.com/properties/1690824/texas-lender-owned-2-hotel-san-antonio-portfolio?recommId=cd75f81443bafcfbee2ae3a5b81a1e3d
{'link': 'https://www.crexi.com/properties/1690824/texas-lender-owned-2-hotel-san-antonio-portfolio', 'address': '80 and 88 NE Interstate 410 Loop, San Antonio, TX 78216', 'Starting Bid:': '$2,500,000', 'Auction Added': 'Oct 23, 2024', 'Time Since Last Update': '8 hours', 'Property Type': 'Hospitality', 'Subtype': 'Hotel', 'Keys': '198', 'Buildings': '2', 'Zoning': 'C-3 General Commercial', 'APN': '12025-000-1220, 12025-00-1230', 'Ownership': 'Fee Simple', 'names': ['Joe Jones PRO', 'Brad Sinclair', 'Christopher Kilcullen PRO', 'Alexsis Wolfson PRO'], 'phones': ['678.628.2290', '770.692.1605', '720.508.8108', '949.799.4658'], 'licenses': ['GA 347019, LA BROK.995716418-ACT', 'GA 326421', 'CO FA100075333', '-']}
2 860 link:  https://www.crexi.com/properties/1699429/texas-conviva-care-center-san-antonio-tx?recommId=cd75f81443bafcfbee2ae3a5b81a1e3

This last cell is for filtering and outputing on in a table format, it also can load from ```.JSON``` file

In [3]:

datajson = json.load(open("data.json"))


df = pd.DataFrame.from_dict(datajson, orient='index')

print("Keywords for filtering: ",df.columns)

df.to_excel("full_scrape.xlsx")

myprops = ['link','Property Type','Subtype','Tenancy', 'Brand/Tenant','Square Footage','Price/Sq Ft (Land Value)', 'Price/Acre (Land Value)','Cap Rate','Occupancy', 'NOI', "Year Built",'Lot Size (acres)','Asking Price:','Starting Bid:','Date Added','Days on Market','Time Since Last Update','names','phones','licenses']

df_filtered = df[myprops]

df_filtered.to_excel("filtered_scrape.xlsx")
print("data saved using the columns: ")
for prop in myprops:
    print(prop)




Keywords for filtering:  Index(['link', 'address', 'Starting Bid:', 'Auction Added',
       'Time Since Last Update', 'Property Type', 'Subtype', 'Keys',
       'Buildings', 'Zoning', 'APN', 'Ownership', 'names', 'phones',
       'licenses', 'Asking Price:', 'Date Added', 'Days on Market',
       'Investment Type', 'Lease Type', 'Tenancy', 'Brand/Tenant',
       'Lease Term', 'Lease Commencement', 'Lease Expiration',
       'Square Footage', 'Cap Rate', 'NOI', 'Year Built', 'Lot Size (acres)',
       'Rent Bumps', 'Broker Co-Op', 'Ground Lease', 'Sale Condition',
       'Tenant Credit', 'Occupancy', 'Parking (spaces)', 'Lease Options',
       'Class', 'Remaining Term', 'Year Renovated', 'Stories', 'Price/Sq Ft',
       'Net Rentable (sq ft)', 'Investment Sub Type', 'Lot Size (sq ft)',
       'Pro-Forma Cap Rate', 'Units', 'Price/Sq Ft (Land Value)',
       'Ceiling Height', 'Loading Docks', 'Number of Tenants',
       'Occupancy Date', 'Dock High Doors', 'Pro-Forma NOI',
       'Parkin